# Question Answering

In [58]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

GOOGLE_API_KEY = "your google api key"
ChatGoogleGenerativeAI.google_api_key = GOOGLE_API_KEY

PINECONE_API_KEY="your pinecone api key"
index_name="ai"

In [14]:
from langchain_pinecone import PineconeVectorStore as lang_pinecone
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

## 1: Q/A Chain

In [41]:


from operator import itemgetter
prompt_str="""
Answer the user question briefly.

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(prompt_str)
question_fetcher=itemgetter("question")
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3,
                             api_key=GOOGLE_API_KEY)


chain = question_fetcher| prompt | llm | StrOutputParser()
query = "what is lahore"  # Question here
response = chain.invoke({"question": query})
print(response)

Lahore is the capital city of the Pakistani province of Punjab.


## 2. Q/A Retriever

In [17]:
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [18]:
embed_model = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')

C:\Users\Asif Computer\AppData\Local\Temp\ipykernel_5260\3555293365.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')


In [19]:
loaders = PyPDFLoader("AI_Engineer_Roadmap.pdf")
pages = loaders.load()

In [21]:
len(pages)

13

In [22]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [23]:
splits = text_splitter.split_documents(pages)

In [24]:
len(splits)

16

In [25]:
# Convert documents into vectors using LangPinecone
vector = lang_pinecone.from_documents(
    splits,             # List of Document objects to be converted into vectors
    embed_model,             # Embedding model used for generating vector representations
    index_name=index_name    # Name of the Pinecone index where vectors will be stored
)

## Chain implementation

In [26]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [50]:
prompt_str="""
Answer the user question based only on the following context:
{context}

Question: {question}
"""
_prompt = ChatPromptTemplate.from_template(prompt_str)
num_chunks=3
retriever = vector.as_retriever(search_type="similarity",
                                        search_kwargs={"k": num_chunks})
chat_llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3,
                             api_key=GOOGLE_API_KEY)
query_fetcher= itemgetter("question")
setup={"question":query_fetcher,"context":query_fetcher | retriever | format_docs}
_chain = (setup |_prompt | chat_llm)

In [51]:
query="classification?"

In [52]:
response=_chain.invoke({"question":query})

In [53]:
response

AIMessage(content='Text classification: Naïve Bayes', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-7a8616ed-77c2-4b2e-95ab-a539438c87fd-0', usage_metadata={'input_tokens': 1051, 'output_tokens': 7, 'total_tokens': 1058})

In [31]:
response.content

'The AI roadmap includes learning technical skills (or tool skills) and soft (or core) skills. The technical skills include coding and math, while the soft skills include research and protection from scams. The total duration of the roadmap is 8 months, with 4 hours of study every day.'

## Conversational Chain

In [54]:
history = []

In [56]:

prompt_str="""
Answer the user question briefly.

Question: {question}

conversation_history: {chat_history}
"""
prompt = ChatPromptTemplate.from_template(prompt_str)
question_fetcher=itemgetter("question")
history_fetcher=itemgetter("chat_history")
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3,
                             api_key=GOOGLE_API_KEY)
setup={"question":question_fetcher,"chat_history":history_fetcher}
chain = setup|prompt | llm | StrOutputParser()
query = "tell me about ai"
response = chain.invoke({"question": query,"chat_history":"\n".join(str(history))})
print(response)
query="user_question:"+query
response="ai_response:"+response
history.append((query, response))


AI, or artificial intelligence, is the simulation of human intelligence processes by machines, especially computer systems. It involves tasks such as learning, reasoning, and problem-solving. AI is used in various fields, including natural language processing, image recognition, and robotics.


In [57]:
history

[('user_question:tell me about lahore',
  'ai_response:Lahore is the capital of the Pakistani province of Punjab and is the second-largest city in Pakistan after Karachi. It is located in the northeast of the country, near the border with India. Lahore is a major cultural, historical, and economic center of Pakistan. The city is home to many historical monuments, including the Lahore Fort, the Badshahi Mosque, and the Shalimar Gardens. Lahore is also a major center of education, with several universities and colleges located in the city.'),
 ('user_question:tell me about ai',
  'ai_response:AI, or artificial intelligence, is the simulation of human intelligence processes by machines, especially computer systems. It involves tasks such as learning, reasoning, and problem-solving. AI is used in various fields, including natural language processing, image recognition, and robotics.')]